In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [6]:
data = pd.read_csv('Dummies.csv')
data.head()

,Avant droite,Passager arrière (à droite),Passager arrière (à gauche),Passager arrière (au milieu),Passager avant (au milieu),Passager central (à gauche),Passager central (au mileu),Passager central (à droite),Conducteur,Piéton,...,3 voies de circulation,4 voies de circulation,5 voies de circulation,6 voies de circulation,7 voies de circulation,8 voies de circulation,9 voies de circulation,10 voies de circulation,11 voies de circulation,12 voies de circulation
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
targets = ['Tué', 'Indemne', 'Blessé léger', 'Blessé hospitalisé']
X = data.drop(targets, axis=1)
X = sm.add_constant(X)
X_opt = X.copy()
Y = data[targets].values.argmax(axis=1)

C:\Users\Erwan\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [8]:
from statsmodels.discrete.discrete_model import MNLogit
significance_level = .05

highest_pvalue = 1
while X_opt.columns.tolist() and highest_pvalue > significance_level:
    result = MNLogit(Y, X).fit()
    print(result.pvalues)
    highest_pvalue = result.pvalues.max()
    less_significant = result.pvalues.index[result.pvalues.values.argmax()]
    if highest_pvalue > significance_level:
        X_opt.drop(inplace=True, columns=less_significant)
        print(f'Eliminated {less_significant} (p-value: {highest_pvalue})')

C:\Users\Erwan\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2290: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\Erwan\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2291: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]


Optimization terminated successfully.
         Current function value: nan
         Iterations 12
                               0   1   2
const                        NaN NaN NaN
Avant droite                 NaN NaN NaN
Passager arrière (à droite)  NaN NaN NaN
Passager arrière (à gauche)  NaN NaN NaN
Passager arrière (au milieu) NaN NaN NaN
...                           ..  ..  ..
8 voies de circulation       NaN NaN NaN
9 voies de circulation       NaN NaN NaN
10 voies de circulation      NaN NaN NaN
11 voies de circulation      NaN NaN NaN
12 voies de circulation      NaN NaN NaN

[234 rows x 3 columns]


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
X_opt

Voir aussi `sklearn.feature_selection.SelectKBest`